# 📰 뉴스 요약 & 필터링 자동화 노트북

`news_summary_email_gemini_filtered.ipynb` 파일입니다.  
- 제목이 '제목 없음' 또는 본문이 '본문을 추출할 수 없습니다.'인 기사는 제외  
- 상위 5개 뉴스를 **Gemini API**로 요약  
- **Gmail SMTP**(SSL)로 HTML 이메일 전송


## ✉️ Gmail SMTP 설정 가이드

아래 단계에 따라 **smtplib**로 메일 전송을 위한 Gmail 환경을 준비하세요.

1. **2단계 인증(2FA)** 활성화  
   - [Google 계정 보안](https://myaccount.google.com/security)에서 설정하세요.

2. **앱 비밀번호(App password)** 발급  
   - 2FA 활성화 후 ‘앱 비밀번호’ 메뉴에서  
     '메일(Mail)' 앱, '기기 선택'을 'Other (Custom name)' → 예: `PythonScript`  
     16자리 비밀번호 생성 후 메모하세요. citeturn0search0

3. **SMTP 서버 정보**  
   | 항목              | 값                  |
   |------------------|--------------------|
   | SMTP 서버 주소     | `smtp.gmail.com`     |
   | 포트 번호 (SSL)    | `465`               |
   | 포트 번호 (TLS)    | `587`               | citeturn0search5

4. **코드 반영 예시**  
   ```python
   import smtplib, ssl
   smtp_server = "smtp.gmail.com"
   port = 465
   sender_email = "you@gmail.com"
   app_password = "앱_비밀번호_16자리"
   context = ssl.create_default_context()
   with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
       server.login(sender_email, app_password)
       server.sendmail(sender_email, receiver_email, message.as_string())
   ```  
   - 앱 비밀번호를 사용해야 Google이 '보안 수준 낮은 앱' 차단을 회피합니다. citeturn0search6


In [ ]:
# -*- coding: utf-8 -*-
\"\"\"
필터링된 뉴스 상위 5개 요약 → Gmail SMTP 전송 코드
필요 패키지:
    pip install google-genai pandas
\"\"\"

import os
import ssl
import smtplib
import pandas as pd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from google import genai
from google.genai import types

# ---------- 사용자 설정 ---------- #
CSV_PATH       = "인공지능_news.csv"   # 뉴스 결과 CSV 경로
SENDER_EMAIL   = "you@gmail.com"       # 보내는 사람 Gmail
RECEIVER_EMAIL = "dest@example.com"    # 받는 사람 이메일
APP_PASSWORD   = "앱_비밀번호_16자리"   # Gmail 앱 비밀번호
GEMINI_API_KEY = "GEMINI_API_KEY"      # Gemini Developer API 키
# ---------------------------------- #

# 1) 뉴스 데이터 로드 및 필터링
df = pd.read_csv(CSV_PATH)
df_clean = df[
    (df['title'].astype(str).str.strip() != '') &
    (df['title'].astype(str).str.strip() != '제목 없음') &
    (df['content'].astype(str).str.strip() != '') &
    (df['content'].astype(str).str.strip() != '본문을 추출할 수 없습니다.')
].copy()
top5 = df_clean.head(5)

if top5.empty:
    raise ValueError("요약할 뉴스가 없습니다.")

# 2) Gemini API 요약
client = genai.Client(api_key=GEMINI_API_KEY)
model = "gemini-2.5-pro-preview-03-25"

summaries = []
for idx, row in top5.iterrows():
    prompt = f\"\"\"다음 뉴스 기사를 한국어로 3문장 이내로 요약해 주세요.\n\n제목: {row['title']}\n본문: {row['content']}\"\"\"
    contents = [types.Content(role="user", parts=[types.Part.from_text(prompt)])]
    cfg = types.GenerateContentConfig(response_mime_type="text/plain")

    summary = ""
    for chunk in client.models.generate_content_stream(model=model, contents=contents, config=cfg):
        summary += chunk.text
    summaries.append(summary.strip())

# 3) 이메일 본문 구성 (HTML)
html_lines = []
for i, (row, summ) in enumerate(zip(top5.itertuples(), summaries), 1):
    html_lines.append(f"<h3>{i}. {row.title}</h3>")
    html_lines.append(f"<p><strong>언론사:</strong> {row.press} | <strong>날짜:</strong> {row.date}</p>")
    html_lines.append(f"<p>{summ}</p>")
    html_lines.append(f"<p><a href='{row.link}'>원문 보기</a></p><hr>")

html_body = "\n".join(html_lines)

msg = MIMEMultipart("alternative")
msg["Subject"] = f"[뉴스 요약] 상위 5개 기사"
msg["From"]    = SENDER_EMAIL
msg["To"]      = RECEIVER_EMAIL
msg.attach(MIMEText(html_body, "html", "utf-8"))

# 4) Gmail SMTP로 전송
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(SENDER_EMAIL, APP_PASSWORD)
    server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, msg.as_string())

print("메일 전송 완료!")